Welcome to the second in a series of educational jupyter notebooks! This series is designed to teach you how to create dynamic spectra and fit scintillation paramters from raw .fits format pulsar data.

The purpose of this second notebook is to plot the dynamic spectra created in the last notebook and fit for the scintillation paramters using two different methods.

I highly reccomend working through this notebook slowly, and using the python tutorial supplied in the python documentation as a syntax reference. While I tried to explain everything I could, I invariably missed something and a little more information never hurt anyone. :)

https://docs.python.org/2/tutorial/index.html

WARNING: These codes are the results of many months of trial and error by an inexperienced python programmer! There are probably better ways of doing things than the way I did it, so if you have any advice for improving this notebook or if you encounter any issues along the way, please feel free to reach out to me at mfl3719@rit.edu or make a pull request at https://github.com/MFLaRose/malcolm

MORE IMPORTANT WARNING!!!: Do not try to run this notebook on more than one observation at a time unless you want to discover new and interesting errors! Please refer to the original code, plotter.py, if you want to plot multiple observations.

Good luck, and have fun!


In [ ]:
#!/usr/bin/env python

# import standard packages
import numpy as np
import scipy.optimize as optimize
import scipy.interpolate as interpolate
# import pypulse stuff
import pypulse as pp
import pypulse.archive as arch
import pypulse.utils as u
import pypulse.dynamicspectrum as DS
import pypulse.functionfit as ffit
# import plotting stuff
import matplotlib.pyplot as plt
import matplotlib.cm as cm
# import admin stuff
import datetime as datetime
import os as os
import subprocess as subproc

firsttime = datetime.datetime.now()

We've seen these statements before! Now that we've also seen how to use a few of them, try finding the documentation for each of the libraries above. Each library's documentation should explain its usage in detail, as well as describe the functions it contains. Good documentation also comes with examples, though hundreds of examples for numpy, scipy, and matplotlib are only a google search away!

Sometimes, while programming, you will encounter a problem that you don't know how to solve, but sounds like something a library you've seen before can do. It is always worth digging to see if someone has written your code before, and more often than not, you'll find exactly what you need in a library's documentation. 

The above libraries are only a small subset of all of those that exist in python! A few extra that might be useful to a future astronomers are astropy, pandas, and tensorflow. Remember that everyone likes to do things in their own way, and more than one library may exist that will solve your problem!

NOTE: You've already encountered this a lot so far, but I thought it was worth explaining. A line that starts with a hashtag (#) is a comment. The computer simply ignored these when it executes the code. It is IMPERATIVE that you comment your code while writing it. It not only improves the comprehension for someone looking at your code for the first time, but it's also handy for you when you want to make some changes a few months down the road! Comments are often used for everything as simple as explicity naming a variable for the programmer to carefully detailing every line of code.

In [ ]:
threshold = 3.5

def zapbin(dspec, threshold):
    # We get the median and standard deviation of the data
    #threshold -= 0.5
    spec_med = np.median(dspec)
    print(spec_med, 'median dspec value')
    spec_std = np.std(dspec)
    bins = []
    # Now we go and zap bins
    row = 0
    for r in dspec:
        col = 0
        for b in r:
            if np.abs(b-spec_med) > threshold*spec_std:
                dspec[row, col] = 0.0
                bins.append([col, row])
            col += 1
        row += 1
    bins = np.array(bins)
    # And we return the spectra
    #print(bins)
    return dspec

def zap_spectra(array,threshold):
    # This zaps the bad subints
    badsubints = []
    y = [np.std(array[:,i]) for i in range(array.shape[1])]
    for i in range(array.shape[1]):
        if np.abs(y[i] - np.median(y)) > threshold*np.std(y):
            array[:,i] = 0
            badsubints.append(i)
    # This zaps the bad channels
    badfchans = []
    y = [np.std(array[i,:]) for i in range(array.shape[0])]
    for i in range(array.shape[0]):
        if np.abs(y[i] - np.median(y)) > threshold*np.std(y):
            array[i,:] = 0
            badfchans.append(i)
    # Option to save the new dynamic spectrum
    #newdspec = dspecfile.split("b32")[0]+"b32.zapped.txt"
    #np.savetxt(newdspec, array)
    # Now we return the zapped dynamic spectrum
    #print(badsubints,badfchans)
    return array

def filefinder(path,ext):
    outlist = []
    for r, d, f in os.walk(path):
        for file in f:
            if ext in file:
                outlist.append(path+file)
    return outlist


As before, we start by defining some useful functions that we will need. This illustrates another case where writing your own functions can be useful. Sometimes a programmer writes a function because it makes future code easier to read. In this example, I think these blocks would be far less illustrative if they occured where we call the function later, even though we only need it once. 

Take some time to look over these functions and imagine what they mean. The first function takes in the entire dynamic spectrum we created with the last notebook. It also accepts a value called threshold, which we defined before the functions. In short, the first function checks every bin in the dynamic spectrum and compares it to the median value of the DS. If the absolute value of this bin is greater than a certain threshold (which we choose!), it sets it to zero instead. The value of threshold we choose is the number of standard deviations away from the mean that we want to cut off. For instance, a smaller value of threshold will result in more of the data being removed. 

The second function is almost exactly the same as the first, but removes RFI with a 'broader brush'. Instead of checking each bin in the dynamic spectra, it walks along both the time and frequency axes and checks the median value of the whole array rather than individual bin values. In short, this function zero-weights any frequency or time channel whose median value was greater than the threshold value for zapping. 

And lastly, filefinder returns to save the day! Full disclosure, I found that block of code on stack exchange! I simply wrote it in function form, and I've used it shamelessley ever since. I will reiterate, while it is always a useful exercise learning how to write your own code, chances are good that someone has done it before. To misquote Picasso, "Good artists borrow, great artists steal." 

WARNING: All jokes aside, if you use code published by anyone else, you should absolutely cite your sources! While this may not matter for a simple stack exchange solution, it is considered poor form, and in some circles, plagiarism, to fail to credit your source. For example, if you use a published python library like numpy, you will need to reference it if you publish a paper about your code!

In [ ]:
rawdatapath = 'RawData/'
txtresultspath = 'IntermediateData/'
plotpath = 'Plots/'
parpath = ''

DStxtfiles = []
zfitsfiles = []

This should look familiar too! A note on variable names, it may be difficult to come up with a variable name in the moment, but it is always worth taking the time to come up with something that is descriptive. Think of how much more confusing this would be if I named the lists list1 and list2!

In [ ]:
DStxtfiles = filefinder(txtresultspath,'.txt')
parpath = filefinder(rawdatapath,'.par')
zfitsfiles = filefinder(txtresultspath,'.zfits')

In [ ]:
# SECTION 1
for i in range(len(DStxtfiles)):
    zfitsfile = DStxtfiles[i].split('sFactor')[0] + '.zfits'
    print(zfitsfile)



    # SECTION 2.1
    dspecdata = np.loadtxt(DStxtfiles[i])
    print(DStxtfiles[i])

    # SECTION 2.2
    outstring = 'psrstat -Qq -c '
    dispmeasure = subproc.check_output(outstring + 'dm ' +  zfitsfile, stderr=subproc.STDOUT,shell=True)
    obslen = float(subproc.check_output(outstring + 'length ' + zfitsfile, stderr=subproc.STDOUT,shell=True))
    numbins = subproc.check_output(outstring + 'nbin ' + zfitsfile, stderr=subproc.STDOUT,shell=True)
    nfbins = subproc.check_output(outstring + 'nchan ' + zfitsfile, stderr=subproc.STDOUT,shell=True)
    bandwidth = float(subproc.check_output(outstring + 'bw '+ zfitsfile, stderr=subproc.STDOUT,shell=True))
    centerfreq = float(subproc.check_output(outstring + 'freq ' +  zfitsfile, stderr=subproc.STDOUT,shell=True))

    minfreq = centerfreq - ((-0.5)*(bandwidth))
    maxfreq = centerfreq + ((-0.5)*(bandwidth))

    obslen_hr = float(obslen)/3600.0
    obslen_min = float(obslen)/60.0

    # Here we call our zapping functions
    dspecdata = zapbin(dspecdata,threshold)
    dspecdata = zap_spectra(dspecdata,threshold)

    # SECTION 3
    ds = DS.DynamicSpectrum(dspecdata, \
                            F = np.linspace(minfreq,maxfreq,np.shape(dspecdata)[0]), Funit='MHz',\
                            T = np.linspace(0.0,obslen,np.shape(dspecdata)[1]), Tunit='sec')

    DynSpec = ds.getData()

    # SECTION 4
    fig = plt.figure(figsize=(6,6))
    ax_dspec = fig.add_subplot(111)
    # Plot the Dynamic Spectrum
    DynSpecPlot = ax_dspec.imshow(DynSpec, cmap = cm.hot, aspect = 'auto', interpolation = 'nearest', extent = [0.0, obslen, maxfreq, minfreq], origin = 'lower')
    # Set axis labels
    ax_dspec.set_xlabel('Time [s]')
    ax_dspec.set_ylabel('Frequency [MHz]')

    # Create Colorbar axis
    ax_dspec.axis(xmin=0.0, xmax=obslen, ymin = minfreq, ymax = maxfreq)

    # Create colorbar
    ax_dspec_cbar = plt.colorbar(DynSpecPlot, ax = ax_dspec, fraction = 0.05, pad = 0.03)
    ax_dspec_cbar.set_label(r"Normalized Power")

    # Correct the inverted y-axis
    plt.gca().invert_yaxis()

    # Save the dynamic spectrum with a new extension
    plt.title('Dynamic Spectrum')
    filename = filename.replace('IntermediateData','Plots')
    plt.savefig(filename+str(sFactor)+'dynSpec.png',format='png')
    plt.show()
    #plt.close()

I apologize in advance for the super long block of code, but there really was no way to break it up. Instead, I have opted for commented sections which I will discuss individually. This whole block reads in the dynamic spectrum from the text file we created, grabs some relevant data from the original files, and plots the dynamic spectrum.

SECTION 1:
First, this uses a very 'non-pythonic' method of using a for loop. The standard usage would be for i in zfitsfiles. In fact, this functions exactly the same way as the standard way. I did the above method because python wasn't my first language, and python's way of handling for loops was foreign to me. I left it here as an instance of old but functional code that could be changed. Optional challenge problem --> Can you make the apporpriate changes to make this more 'pythonic'?

The first line within the loop saves each filename in zfitsfiles as a new variable zfitsfile. This is really just for convenience, and made writing this code a little easier.

SECTION 2: 
Loops within loops! If you remember from the first program, we created a bunch of dynamic spectra with different frequency scrunching factors for each observation. If you think of the original observation as the 'base' file, the superceeding for loop looks through a list of those observations, and the nested while loop looks at each dynamic spectrum for each observation. 

SECTION 2.1:
Now we need to do a little bookkeeping. This would be a good place to try out a few print statements if you want to see what goes on under the hood! I've left one in here for you that references which dynamic spectrum we are trying to create. Very useful for bug-fixing!

This piece of code uses the native python replace function. This accepts a string, looks for a given substring, and replaces it with another. In this case, we are changing the filename from the one appropriate for the .zfits file to the one that references the .txt files for that observation. There are probably better ways of doing this, but this felt the safest to me at the time of writing!

We also use numpy's loadtxt to get the data from the text file. You can open these text files to see what the data looks like!

SECTION 2.2:
We need to get some relevant information from the original observation for each dynamic spectrum. We do this with PSRCHIVE's psrstat, which reads the header of the provided .fits file. Note that the bandwidth is often provided as a negative value (why?), so we account for that while calculating minfreq and maxfreq. 

The length of the observation is, by standard, in seconds, so we convert that to minutes and hours for convenience. 

SECTION 3:
This short section of code turns the data from a numpy array (which we called dspecdata) in to a PyPulse DynamicSpectrum class. (More on that here: https://mtlam.github.io/PyPulse/dynamicspectrum.html) This relies on the numpy function linspace, which creates a list from the first argument to the second, using the spacing provided by the third argument. Here, np.shape provides the length of the original axis, so our spacing remains the same as the original data. 

ASSIDE: You will almost never write a program that doesn't use numpy in some way. I highly recommend familiarizing yourself with some of the most commonly used functions. In fact, numpy's website contains many useful tutorials and guides which you may find useful later! https://numpy.org/doc/stable/

SECTION 4:
Now we can finally plot the dynamic spectrum! This next section relies almost entirely on matplotlib, which is an incredibly robust plotting library. 

NOTE FOR PROOFREADERS: Matplotlib is my weakest area, if you have any suggestions for improving this section, either code or written word, please let me know!

In [ ]:
 for i in range(len(DStxtfiles)):
    zfitsfile = DStxtfiles[i].split('sFactor')[0] + '.zfits'
    print(zfitsfile)
    
    ### Reinitialize the data
    dspecdata = np.loadtxt(DStxtfiles[i])
    print(DStxtfiles[i])

    # SECTION 2.2
    outstring = 'psrstat -Qq -c '
    dispmeasure = subproc.check_output(outstring + 'dm ' +  zfitsfile, stderr=subproc.STDOUT,shell=True)
    obslen = float(subproc.check_output(outstring + 'length ' + zfitsfile, stderr=subproc.STDOUT,shell=True))
    numbins = subproc.check_output(outstring + 'nbin ' + zfitsfile, stderr=subproc.STDOUT,shell=True)
    nfbins = subproc.check_output(outstring + 'nchan ' + zfitsfile, stderr=subproc.STDOUT,shell=True)
    bandwidth = float(subproc.check_output(outstring + 'bw '+ zfitsfile, stderr=subproc.STDOUT,shell=True))
    centerfreq = float(subproc.check_output(outstring + 'freq ' +  zfitsfile, stderr=subproc.STDOUT,shell=True))

    minfreq = centerfreq - ((-0.5)*(bandwidth))
    maxfreq = centerfreq + ((-0.5)*(bandwidth))

    obslen_hr = float(obslen)/3600.0
    obslen_min = float(obslen)/60.0

    # Here we call our zapping functions
    dspecdata = zapbin(dspecdata,threshold)
    dspecdata = zap_spectra(dspecdata,threshold)

    # SECTION 3
    ds = DS.DynamicSpectrum(dspecdata, \
                            F = np.linspace(minfreq,maxfreq,np.shape(dspecdata)[0]), Funit='MHz',\
                            T = np.linspace(0.0,obslen,np.shape(dspecdata)[1]), Tunit='sec')

    DynSpec = ds.getData()
    
    
    ### Produce secondary spectrum
    secondspec = ds.secondary_spectrum(log=True)

    bw = ds.getBandwidth()
    chunkDuration = ds.dT
    nchans = len(ds.F)
    conj_time_max = np.multiply(np.divide(1.0, np.multiply(2.0, chunkDuration)), \
                                1000.0)
    conj_time_min = -conj_time_max
    conj_freq_max = np.divide(nchans, np.multiply(2.0, bw))
    conj_freq_min = -conj_freq_max


    fig = plt.figure(figsize=(6,6))
    ax_secondspec = fig.add_subplot(111)
    secspecplot = ax_secondspec.imshow(secondspec/np.max(secondspec), \
                                       cmap = cm.viridis, aspect = 'auto', \
                                       extent=[conj_time_min, conj_time_max, conj_freq_min, conj_freq_max])
    ax_secondspec.set_ylabel(r"Differential Delay (us)")
    ax_secondspec.set_xlabel(r"Differential Doppler Frequency (mHz)")

    ax_secondspec.axis(xmin=conj_time_min, xmax=conj_time_max, ymin=0.0, \
                       ymax=conj_freq_max)

    secspec_cbar = plt.colorbar(secspecplot, ax = ax_secondspec, \
                                fraction = 0.05, pad = 0.03)
    secspec_cbar.set_label(r"Normalized Power")
    plt.title('Secondary Spectrum')
    plt.tight_layout()
    plt.savefig(filename+str(sFactor)+"Secondary.png",format='png')
    plt.show()

Now we can create the secondary spectrum. This is almost equivalent to a power spectrum, which describes the distribution of power over frequency in a time series. However, the secondary spectrum for a pulsar describes how the pulse time of arrival has changed over frequency due to scattering. 

Once again, PyPulse takes care of the heavy lifting for us, and we are left with an array that we need to plot. 

In [ ]:
 ### SECTION 1
for i in range(len(DStxtfiles)):
    zfitsfile = DStxtfiles[i].split('sFactor')[0] + '.zfits'
    print(zfitsfile)
    
    ### Reinitialize the data
    dspecdata = np.loadtxt(DStxtfiles[i])
    print(DStxtfiles[i])

    # SECTION 2.2
    outstring = 'psrstat -Qq -c '
    dispmeasure = subproc.check_output(outstring + 'dm ' +  zfitsfile, stderr=subproc.STDOUT,shell=True)
    obslen = float(subproc.check_output(outstring + 'length ' + zfitsfile, stderr=subproc.STDOUT,shell=True))
    numbins = subproc.check_output(outstring + 'nbin ' + zfitsfile, stderr=subproc.STDOUT,shell=True)
    nfbins = subproc.check_output(outstring + 'nchan ' + zfitsfile, stderr=subproc.STDOUT,shell=True)
    bandwidth = float(subproc.check_output(outstring + 'bw '+ zfitsfile, stderr=subproc.STDOUT,shell=True))
    centerfreq = float(subproc.check_output(outstring + 'freq ' +  zfitsfile, stderr=subproc.STDOUT,shell=True))

    minfreq = centerfreq - ((-0.5)*(bandwidth))
    maxfreq = centerfreq + ((-0.5)*(bandwidth))

    obslen_hr = float(obslen)/3600.0
    obslen_min = float(obslen)/60.0

    # Here we call our zapping functions
    dspecdata = zapbin(dspecdata,threshold)
    dspecdata = zap_spectra(dspecdata,threshold)

    # SECTION 3
    ds = DS.DynamicSpectrum(dspecdata, \
                            F = np.linspace(minfreq,maxfreq,np.shape(dspecdata)[0]), Funit='MHz',\
                            T = np.linspace(0.0,obslen,np.shape(dspecdata)[1]), Tunit='sec')

    DynSpec = ds.getData()


    ### Produce ACF
    fig = plt.figure(figsize=(6,6))

    acf2d = ds.acf2d()
    ACF_2D = ds.acf
    plotacf = ds.getACF()
    ax_acf = fig.add_subplot(111) # Trying this...
    acfPlot = ax_acf.imshow(plotacf, cmap=cm.hot, origin = 'lower', aspect = 'auto', \
                            interpolation = 'nearest')
    plt.tight_layout()
    ax_acf.set_xlabel("Time [bins]")
    ax_acf.set_ylabel("Frequency [bins]")
    plt.title('Autocorrelation')
    plt.savefig(filename+str(sFactor)+'Autocorrelation.png',format='png')
    plt.show()
    #plt.close()

    ### SECTION 2
    # CAN CHANGE DIVISOR (4) FOR BETTER FIT
    #print(filename)
    maxr = np.shape(ds.getData())[0]/4 # frequency shape
    maxc = np.shape(ds.getData())[1]/4 # time shape

    delta_t_d, err_t_d, delta_nu_d, err_nu_d \
    = ds.scintillation_parameters(simple=True, full_output=True, show = False, \
                                  maxr=maxr, maxc=maxc, cmap=cm.hot)

    acfparamlist = []
    acfparamlist.append("delta_t_d: "+str(delta_t_d)+"\n")
    acfparamlist.append("err_t_d: "+str(err_t_d)+"\n")
    acfparamlist.append("delta_nu_d: "+str(delta_nu_d)+"\n")
    acfparamlist.append("err_nu_d: "+str(err_nu_d)+"\n")
    #acfparamlist.append("rotation: "+str(rotation)+"\n")
    #acfparamlist.append("err_rot: "+str(err_rot))

    #acfparamarray = np.array(acfparamlist)
    print(acfparamlist)
    acffile = filename+str(sFactor)+'acfparam.txt'

    try:
        subprocess.call('touch '+acffile,shell=True)

    except:
        print("Something's wrong...")

    acffileopen = open(filename+str(sFactor)+'acfparam.txt','w')


    acffileopen.writelines(acfparamlist)
    acffileopen.close()

Now we perform an autocorellation function on the dynamic spectrum. If you've never heard of this before, it probably sounds terrifying! In reality, all it is doing is making a copy of the dynamic spectrum, sliding the copy over the original, and calculating the integral of the product of the two. Maybe that wasn't the best explanation, but wikipedia isn't the most helpful here. I've pasted some resources below to try to aid in your understanding.
https://en.wikipedia.org/wiki/Autocorrelation
https://en.wikipedia.org/wiki/Cross-correlation#Explanation

PyPulse needs the ACF to fit the scintillation parameters, but it actually does this internally, like how we created the dynamic and secondary spectra. Here, we plot the ACF to make sure something didn't go horribly wrong. Note that we didn't actually use the ACFs produced on lines 8 and 9. These will be used in the next block of code, however, which brings us to an important point about jupyter notebooks. While I never explicity stated it, I think it may be plain to see by now that each cell in a jupyter notebook isn't independent from the others. If I declare a variable in one cell, I can always use it in another without re-declaring it. Jupyter notebooks are a really helpful bugfixing and educational tool, however for functional code, it is often better to condense everything into a single python script.  

Section 1 is almost a direct copy of the code used in the other sections for creating plots, so I will skip explaining it again.

Section 2 does things a little differently. So far, we've been using lists or arrays of numbers to generate other lists or arrays of numbers. For example, this is exactly what we did during folding! Now we want to be able to characterize the scintillation with only a few numbers, and we do that by fitting a gaussian function to the center of the ACF. Our two observables, delta_t_d and delta_nu_d, are the half width at e^-1 and HWHM of the gaussian in their respective directions. <-- BAD EXPLANATION, NEEDS WORK

We then need to save these values along with their respective errors somewhere. It doesn't make sense to put these in a plot, so we'll write them to a text file instead. We first give it a 'unique' filename by simply adding on 'acfparam.txt' to the original filename, and we 'try' to create a file to dump the values in. 

NOTE: All of this should theoretically be doable in much less code, but I couldn't figure out a way to do it. 

The next section of code was written by a PhD student at WVU, Brent Shapiro-Albert. It's purpose is the same as the above section, but its all written out explicity so you can see what goes on under the hood. He was even kind enough to leave descriptive comments, woohoo!

In [ ]:
for i in range(len(DStxtfiles)):
    zfitsfile = DStxtfiles[i].split('sFactor')[0] + '.zfits'
    print(zfitsfile)
    
    ### Reinitialize the data
    dspecdata = np.loadtxt(DStxtfiles[i])
    print(DStxtfiles[i])

    # SECTION 2.2
    outstring = 'psrstat -Qq -c '
    dispmeasure = subproc.check_output(outstring + 'dm ' +  zfitsfile, stderr=subproc.STDOUT,shell=True)
    obslen = float(subproc.check_output(outstring + 'length ' + zfitsfile, stderr=subproc.STDOUT,shell=True))
    numbins = subproc.check_output(outstring + 'nbin ' + zfitsfile, stderr=subproc.STDOUT,shell=True)
    nfbins = subproc.check_output(outstring + 'nchan ' + zfitsfile, stderr=subproc.STDOUT,shell=True)
    bandwidth = float(subproc.check_output(outstring + 'bw '+ zfitsfile, stderr=subproc.STDOUT,shell=True))
    centerfreq = float(subproc.check_output(outstring + 'freq ' +  zfitsfile, stderr=subproc.STDOUT,shell=True))

    minfreq = centerfreq - ((-0.5)*(bandwidth))
    maxfreq = centerfreq + ((-0.5)*(bandwidth))

    obslen_hr = float(obslen)/3600.0
    obslen_min = float(obslen)/60.0

    # Here we call our zapping functions
    dspecdata = zapbin(dspecdata,threshold)
    dspecdata = zap_spectra(dspecdata,threshold)

    # SECTION 3
    ds = DS.DynamicSpectrum(dspecdata, \
                            F = np.linspace(minfreq,maxfreq,np.shape(dspecdata)[0]), Funit='MHz',\
                            T = np.linspace(0.0,obslen,np.shape(dspecdata)[1]), Tunit='sec')
    
    acf2d = ds.acf2d()
    ACF_2D = ds.acf

    DynSpec = ds.getData()


    # Brent's Method
    # Define the initial figure
    fig = plt.figure(figsize = (6,6)) # NEW EDIT HERE
    ax_1Dfit_up = fig.add_subplot(211)
    ax_1Dfit_down = fig.add_subplot(212)
    # Define a function that returns a gaussian, will be use dlater
    def Gfit(x, p):
        return p[0] * np.exp(-((x-p[1])/(np.sqrt(2)*p[2]))**2) + p[3]

    # We need to get a few values from the spectra
    dT = ds.dT # size of time bin, units as input in DS.dynamicspectrum call (here minutes)
    dF = ds.dF # same as dT but the size of the frequency channels (here MHz)
    acfshape = np.shape(ds.acf) # array dimensions of the 2-D ACF
    eta = 0.2 # parameter used to determine scintillation parameters
    NF = len(ds.F) # Number of frequency channels
    Faxis = (np.arange(-(NF-1),NF,dtype=np.float)*np.abs(dF)) # frequency value of each channel
    NT = len(ds.T) # number of time bins
    Taxis = (np.arange(-(NT-1),NT,dtype=np.float)*np.abs(dT))[1:-1] # time value of each time bin
    plotbound = 1.0

    centerrind = np.where(Faxis == 0)[0] # index where frequency lag is 0, or center index in frequency lag
    centercind = np.where(Taxis == 0)[0] # as above but for time lag

    # Now we sum across the frequency axis
    Faxis_summed = np.sum(ds.acf, axis=0) # for timescale
    Faxis_summed = Faxis_summed[1:-1] # cut edges, they're usually bad values
    #print('Lookie!',Faxis_summed)
    # Now sum across the time axis
    Taxis_summed = np.sum(ds.acf, axis=1) # for frequency scale
    FnormFac = np.max(Faxis_summed) # get normalizing factor for fitting later
    TnormFac = np.max(Taxis_summed) # as above
    #print('Lookie2!',Taxis_summed)

    # We change the fitting regions to be the same number of bins as in the 2D fit
    # This can be varied a bit but helps better fit things and give higher consistancy
    low_t_bound = int(centercind-plotbound*maxc+1)
    high_t_bound = int(centercind+plotbound*maxc+1)
    low_f_bound = int(centerrind-plotbound*maxr+1)
    high_f_bound = int(centerrind+plotbound*maxr)
    # Make sure we don't exceed bounds of ACF -> probably not important here, but won't slow things down
    if low_t_bound < 0:
        low_t_bound = 0
    if high_t_bound > len(Taxis):
        high_t_bound = len(Taxis)
    if low_f_bound < 0:
        low_f_bound = 0
    if high_f_bound > len(Faxis):
        high_f_bound = len(Faxis)
    # regardless of bounds, we set the edges of the 1-D ACF to be the minimum value in that range as this
    # helps to better fit a gaussian. This can be played with as well to obtain better fits
    quarter_t_bins = int(np.floor(len(Taxis[low_t_bound:high_t_bound])/8))
    # This is to get a separate array for fitting
    Faxis_cor = []
    for v in Faxis_summed[low_t_bound:high_t_bound]:
        Faxis_cor.append(v)
    Faxis_cor = np.array(Faxis_cor)
        # We replace the values with the appropriate things

    print(low_t_bound,high_t_bound)
    print(Faxis_summed)
    print(Faxis_summed[low_t_bound:high_t_bound]) ### NEW PRINT STATEMENT
    f_rep_val = np.min(Faxis_summed[low_t_bound:high_t_bound])
    Faxis_cor[:quarter_t_bins] = f_rep_val
    Faxis_cor[len(Faxis_summed[low_t_bound:high_t_bound])-quarter_t_bins:] = f_rep_val
    # fit for scintillation timescale along axis better for fitting
    pout, errs = ffit.gaussianfit(Taxis[low_t_bound:high_t_bound],Faxis_cor,baseline=True)
    f = interpolate.interp1d(Taxis,ffit.funcgaussian(pout,Taxis,baseline=True)-(pout[3]+pout[0]/np.e))
    # plot stuff
    # for timescale
    ax_1Dfit_up.plot(Taxis[low_t_bound:high_t_bound], Faxis_summed[low_t_bound:high_t_bound]/FnormFac, c = 'k')
    ax_1Dfit_up.plot(Taxis[low_t_bound:high_t_bound], Gfit(Taxis[low_t_bound:high_t_bound], pout)/FnormFac, c= 'r')
    #ax_1Dfit_up.plot(Taxis[low_t_bound:high_t_bound], Faxis_cor/FnormFac, c = 'g') # this is the line that is fit to
    # This actually finds what the scintillation timescale is, as opposed to just fitting the functions used to find this
    try:
        delta_t_d = optimize.brentq(f,0,Taxis[-1])
        # If this fails we assume the timescale cannot be resolved and set it to the length of the observation
    except:
        print("Error with delta_t_d, setting to max value")
        delta_t_d = dT*NT
    # more plot labeling
    ax_1Dfit_up.set_xlabel(r"Time Lag (s)")
    ax_1Dfit_up.set_ylabel(r"Normalized Power")
    ax_1Dfit_up.set_xlim([Taxis[low_t_bound], Taxis[high_t_bound-1]])
    ax_1Dfit_up.set_ylim([np.min(Faxis_summed[low_t_bound:high_t_bound])/FnormFac-0.05, 1.0])

    # for scintillation bandwidth

    # We replace the edge values with zero to better fit things
    quarter_f_bins = int(np.floor(len(Faxis[low_f_bound:high_f_bound])/8))
    Taxis_cor = []
    for v in Taxis_summed[low_f_bound:high_f_bound]:
        Taxis_cor.append(v)
    Taxis_cor = np.array(Taxis_cor)

    t_rep_val = np.min(Taxis_summed[low_f_bound:high_f_bound])
    Taxis_cor[:quarter_f_bins] = t_rep_val
    Taxis_cor[len(Taxis_summed[low_f_bound:high_f_bound])-quarter_f_bins:] = t_rep_val


    # And do some plotting for this
    ax_1Dfit_down.plot(Faxis[low_f_bound:high_f_bound], Taxis_summed[low_f_bound:high_f_bound]/TnormFac, c = 'k')
    # fit for scintilattion bandwidth
    pout, errs = ffit.gaussianfit(Faxis[low_f_bound:high_f_bound],Taxis_cor,baseline=True)
    f = interpolate.interp1d(Faxis,ffit.funcgaussian(pout,Faxis,baseline=True)-(pout[3]+pout[0]/2))

    ax_1Dfit_down.plot(Faxis[low_f_bound:high_f_bound], Gfit(Faxis[low_f_bound:high_f_bound], pout)/TnormFac, c= 'r')
    #ax_1Dfit_down.plot(Faxis[low_f_bound:high_f_bound], Taxis_cor/TnormFac, c= 'g') # This shows what we used to fit the Gaussian
    # Now actually try to get the scintillation bandwidth
    try:
        delta_nu_d = optimize.brentq(f,0,Faxis[-1])
        # And if it fails assume it's again, unresolvable
    except:
        print "Error with delta_nu_d, setting to max value"
        delta_nu_d = dF*NF
    # more plotting things...
    ax_1Dfit_down.set_xlabel(r"Frequency Lag (MHz)")
    ax_1Dfit_down.set_ylabel(r"Normalized Power")
    ax_1Dfit_down.set_xlim([Faxis[low_f_bound], Faxis[high_f_bound-1]])
    ax_1Dfit_down.set_ylim([np.min(Taxis_summed[low_f_bound:high_f_bound]/TnormFac)-0.05, 1.0])
    # We estimate the errors on the scintillation parameters
    bw = ds.getBandwidth()
    T = ds.getTspan()
    if delta_t_d == 0.0:
        N_d = (1+eta * bw/delta_nu_d)
    elif delta_nu_d == 0.0:
        N_d = (1+eta*T/delta_t_d)
    else:
        N_d = (1+eta * bw/delta_nu_d) * (1+eta*T/delta_t_d)
    fse_nu_d = delta_nu_d/(2*np.log(2)*np.sqrt(N_d)) #log because of FWHM?
    fse_t_d = delta_t_d/(2*np.sqrt(N_d))

    err_nu_d = fse_nu_d
    err_t_d = fse_t_d 

    # And we add these values to the plots to see them easily    
    try:
        scinttxt = r"$\Delta t_{d}$  = %.3f $\pm$ %.3f seconds" % (delta_t_d, err_t_d)
        ax_1Dfit_up.text(0.98, 0.95, scinttxt, ha='right', va='top', color = 'k', \
                         transform=ax_1Dfit_up.transAxes, fontsize=12)
    except:
        print("Error with scintillation timescale")
    try:
        scinttxt = r"$\Delta \nu_{d}$  = %.3f $\pm$ %.3f MHz" % (delta_nu_d, err_nu_d)
        ax_1Dfit_down.text(0.95, 0.95, scinttxt, ha='right', va='top', color = 'k', \
                           transform=ax_1Dfit_down.transAxes, fontsize=12)
    except:
        print("Error with scintillation bandwidth")


    plt.savefig(filename+'Values.png',format='png')
    print(filename)
    plt.show()    
print('Success!')

Whew! If you've made it this far without giving up, I applaud your perseverence! In truth, you probably don't need to understand the above block of code perfectly, unless it is creating errors. 